# Importing Libraries

In [ ]:
# External imports
import pandas as pd
import numpy as np
from PIL import Image, ImageDraw, ImageFont
from PIL.PngImagePlugin import PngImageFile
import requests
from textwrap import wrap

In [ ]:
# Internal imports
from utils import import_xml_as_df

# 1. Importing XML as Pandas DataFrame

In [ ]:
# Generate raw offers dataframe
df_offers_raw = import_xml_as_df(path='../resources/examples/LomadeeDownload_raw.xml')

In [ ]:
# Display raw offers dataframe
display(df_offers_raw)

# 2. Refining DataFrame

In [ ]:
# Refine Dataframe (Social Soul) function
def refine_dataframe(df:pd.core.frame.DataFrame) -> pd.core.frame.DataFrame:
    
    # STEP 1: Select columns of interest
    # Set columns of interest
    columns_of_interest = [
        'offerName',
        #'sellerId',
        #'sellerThumbnail',
        'offerLink',
        'offerThumbnail',
        'priceFrom',
        #'sellerName',
        'priceTo',
        #'sku',
        #'categoryName',
        #'categoryId',
    ]
    # Select columns of interest only
    df = df.loc[:,columns_of_interest]
    
    
    # STEP 2: Cast prices from STR to FLOAT64
    df['priceFrom'] = df['priceFrom'].astype('float64')
    df['priceTo'] = df['priceTo'].astype('float64')
    
    
    # STEP 3: Clean absent values
    # Replace NaN (Not a Number) with None
    df = df.replace(np.nan, None)
    
    
    # Return refined dataframe
    return df

In [ ]:
# Generate refined offers dataframe
df_offers_refined = refine_dataframe(df=df_offers_raw)

In [ ]:
# Display refined offers dataframe
display(df_offers_refined)

# 3. Creating Instagram Stories images

In [ ]:
# Add Text To Image function
def add_text_to_image(
    im:PngImageFile,
    font:str = '../resources/fonts/OpenSans-VariableFont_wdth,wght.ttf',
    font_size:int = 12,
    x:int = 0,
    y:int = 0,
    text:str = 'No text provided',
    text_align:str = 'left',
):
    
    # Get a drawing context
    draw = ImageDraw.Draw(im=im)
    
    # Set font
    text_font = ImageFont.truetype(
        font=font,
        size=font_size,
        encoding='unic'
    )
    
    # Add text to image
    draw.multiline_text(
        xy=(x,y),
        text=text,
        fill=(0,0,0),
        font=text_font,
        align=text_align,
        stroke_width=0
    )
    
    # Return image (with added text)
    return im
    

In [ ]:
def generate_ig_stories_image(
    offer_thumbnail:str = None,
    offer_name:str = None,
    offer_price_from:str = None,
    offer_price_to:str = None,
    file_path:str = None
) -> str:
    
    # STEP 1: Create empty IG Stories image from template
    # Create empty story from template
    im_ig_stories = Image.open(fp='../resources/story_template_720x1280_final.png')
    
    
    # STEP 2: Add offer thumbnail to IG Stories image
    # Get offer thumbnail
    im_thumbnail = Image.open(requests.get(offer_thumbnail, stream=True).raw)
    # Resize offer thumbnail
    im_thumbnail = im_thumbnail.resize(size=(640,640))
    # Paste offer thumbnail to story image
    im_ig_stories.paste(im=im_thumbnail,box=(40, 130))
    
    
    # STEP 3: Add offer name to IG Stories image
    # Add offer name
    im_ig_stories = add_text_to_image(
        im=im_ig_stories,
        font_size=40,
        x=27.5, # (10+17.5)
        y=827.5, # (100+700+20+7.5)
        text= '\n'.join(wrap(text=offer_name, width=30)),
        text_align='left'
    )
    
    
    # STEP 4: Add offer price to IG Stories image
    # If no priceFrom:
    if offer_price_from == None:
        price_text = f'Por apenas R${offer_price_to:.2f}!'
    # Else (if priceFrom):
    else:
        price_text = f'De R${offer_price_from:.2f} por apenas R${offer_price_to:.2f}!'
    # Add offer price to IG Stories image
    im_ig_stories = add_text_to_image(
        im=im_ig_stories,
        font_size=37.5,
        x=27.5, # (10+17.5)
        y=1062.5, # (100+700+20+210+20+12.5)
        text=price_text,
        text_align='left'
    )
    
    
    # STEP 5: Save IG Stories image to temp folder
    im_ig_stories.save(fp=file_path, format='png')
    
    
    # Return path fo IG Stories image
    return file_path
    

In [ ]:
# Add Instagram Stories Images To Dataframe function
def add_ig_stories_images_to_df(df_offers:pd.core.frame.DataFrame) -> pd.core.frame.DataFrame:
    
    # Create empty list of IG Stories images
    ig_stories_images = []
    
    # For each offer in offers dataframe:
    for i in range(len(df_offers)):
        
        # Generate IG Stories image
        ig_stories_images.append(
            generate_ig_stories_image(
                offer_thumbnail= df_offers.loc[i, 'offerThumbnail'],
                offer_name= df_offers.loc[i, 'offerName'],
                offer_price_from= df_offers.loc[i, 'priceFrom'],
                offer_price_to= df_offers.loc[i, 'priceTo'],
                file_path= f'../temp/ig_stories_images/offer_{i}.png',
            )
        )
    
    # Add IG Stories images list as column in dataframe        
    df_offers['storiesImage'] = ig_stories_images
    
    # Return dataframe
    return df_offers

In [ ]:
df_offers_ready = add_ig_stories_images_to_df(df_offers=df_offers_refined)

In [ ]:
display(df_offers_refined)